# Final Neural Network Model Selection Process

In [84]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, r2_score
from sklearn.metrics import mean_squared_error as mse

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import keras_tuner as kt

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
from  IPython import display

import pathlib
import shutil
import tempfile

print(tf.__version__)

2.7.0


Ensure the random state is consistent:

In [85]:
from numpy.random import seed
from numpy.random import RandomState

In [86]:
np.random.seed(42)
tf.random.set_seed(42)

### Data loading, transformation, and normalization

In [87]:
data = pd.read_csv("./data/nba_final_data.csv")
data = data.sample(frac=1, random_state = 42) # Shuffle data
y = data['PLUS_MINUS_HOME']
X = data.drop(['GAME_ID', "TEAM_ID_HOME", "TEAM_ID_AWAY", "GAME_DATE", "SEASON",
               "PLUS_MINUS_HOME", "MIN_HOME", 'WL_Home_modified'], axis=1)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

### HyperParameter Tuning With Bayesian Optimization: Setup

In [88]:
# Define log dir 
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [89]:
def model_builder(hp):
    model = keras.Sequential()
    
    # hyperparameter boolean for performing dropout 
    dropout = hp.Boolean("dropout") 
    # hyperparameter for percent of units to dropout 
    if dropout:
        drop_percent = hp.Choice("drop_percent", [0.05, 0.1, 0.25, 0.5])
        
    kernel_regularizer= regularizers.l2(0.001)
    
    # hyperparameter for choice of regularization strength
    regularization = hp.Choice("regularization_strength", [0.0001, 0.001, 0.01, 0.1, 0.25, 0.5])
    
    model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))
    for i in range(hp.Int("num_layers", 1, 3)):
        hp_units = hp.Int('units', min_value=8, max_value=64, step=4)
        model.add(
            keras.layers.Dense(units=hp_units,
                               activation='elu',
                               kernel_regularizer = regularizers.l2(regularization))
        )
        # Add dropout layer if dropout hyperparameter is True
        if dropout:
            keras.layers.Dropout(drop_percent)
            
    model.add(keras.layers.Dense(1)) # output layer
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
            loss='mse',
            metrics=['mse'])
    return model

In [90]:
def get_callbacks(name):
    return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25),
    tf.keras.callbacks.TensorBoard(logdir/name)
    ]

In [91]:
tuner = kt.BayesianOptimization(model_builder,
                                objective='val_mse',
                                max_trials = 100,
                                seed=42,
                                overwrite=True
                                )

### HyperParameter Tuning With Bayesian Optimization: Search for Parameters
Here, we search for the best set of hyperparameters with the tuner, extract the hyperparemeters, and build a model with the results 

In [92]:
tuner.search(X_train, y_train, epochs=200, validation_split=0.2, callbacks=get_callbacks('tuner'))

Trial 100 Complete [00h 01m 11s]
val_mse: 149.70858764648438

Best val_mse So Far: 148.43832397460938
Total elapsed time: 01h 53m 49s
INFO:tensorflow:Oracle triggered exit


In [93]:
# Get the optimal hyperparameter model
best_hps = tuner.get_best_hyperparameters()[0]

# Build and save it for future use 
best_model = tuner.hypermodel.build(best_hps)

In [94]:
print(f"""
The hyperparameter search is complete.
Num_layers: {best_hps.get('num_layers')}
Num_units: {best_hps.get('units')}
Dropout: {best_hps.get('dropout')}
Dropout rate: {best_hps.get('drop_percent')}
Regularization strength: {best_hps.get("regularization_strength")}
Learning Rate: {best_hps.get('learning_rate')}
""")


The hyperparameter search is complete.
Num_layers: 1
Num_units: 36
Dropout: False
Dropout rate: 0.25
Regularization strength: 0.25
Learning Rate: 0.01



### HyperParameter Tuning With Bayesian Optimization: Refit and Epoch Selection
Now that we have the best hyperparameters, we refit the model. To determine the optimal number of training epochs, we find the epoch number of the best results as determined by mean square error on the validation set. Finally, we rebuild the model from the hyperparameters, specify the number of epochs as determined by the validation mean square error, and refit the model with the full training data set (train + validation). Then, test results are reported in the final cell. 

In [95]:
history = best_model.fit(X_train, y_train,
                            epochs=300,
                            validation_split=0.2,
                            verbose=0,
                            callbacks=get_callbacks('best_model_epoch_selection'))


Epoch: 0, loss:166.2021,  mse:160.9939,  val_loss:158.9273,  val_mse:155.1157,  
..................................................

In [96]:
validation_mse = history.history['val_mse']
min_mse = min(validation_mse)
min_idx = validation_mse.index(min_mse)
num_epochs = min_idx
print(f"Minimum MSE: {min_mse}")

Minimum MSE: 148.5083465576172


In [97]:
best_model = tuner.hypermodel.build(best_hps)
history=best_model.fit(X_train, y_train, epochs=num_epochs, callbacks=get_callbacks('best_model'))
best_model.save('final_model')

Epoch 1/38
406/416 [============================>.] - ETA: 0s - loss: 163.7635 - mse: 158.9777
Epoch: 0, loss:163.8388,  mse:159.0725,  
416/416 [==============================] - 1s 766us/step - loss: 163.8388 - mse: 159.0725
Epoch 2/38
416/416 [==============================] - 0s 916us/step - loss: 160.6180 - mse: 157.1347
Epoch 3/38
416/416 [==============================] - 0s 776us/step - loss: 159.4421 - mse: 156.2965
Epoch 4/38
416/416 [==============================] - 0s 782us/step - loss: 158.9559 - mse: 155.8466
Epoch 5/38
416/416 [==============================] - 0s 731us/step - loss: 158.3138 - mse: 155.3365
Epoch 6/38
416/416 [==============================] - 0s 710us/step - loss: 157.7860 - mse: 154.9476
Epoch 7/38
416/416 [==============================] - 0s 721us/step - loss: 157.2583 - mse: 154.5086
Epoch 8/38
416/416 [==============================] - 0s 780us/step - loss: 157.4098 - mse: 154.4004
Epoch 9/38
416/416 [==============================] - 0s 752us/ste

In [98]:
test_results = best_model.evaluate(X_test, y_test)
dict(zip(best_model.metrics_names, test_results))

104/104 [==============================] - 0s 484us/step - loss: 154.7836 - mse: 152.3198


{'loss': 154.7836151123047, 'mse': 152.31983947753906}

In [99]:
pred = best_model.predict(X_test)
pred - np.array(y_test)

array([[ -0.577,  -9.577,  20.423, ...,  14.423,   5.423, -13.577],
       [ -7.921, -16.921,  13.079, ...,   7.079,  -1.921, -20.921],
       [-10.2  , -19.2  ,  10.8  , ...,   4.8  ,  -4.2  , -23.2  ],
       ...,
       [-14.795, -23.795,   6.205, ...,   0.205,  -8.795, -27.795],
       [-13.456, -22.456,   7.544, ...,   1.544,  -7.456, -26.456],
       [-14.151, -23.151,   6.849, ...,   0.849,  -8.151, -27.151]])

In [100]:
def model_evaluation(y_pred, y_actual):
    """
    returns evaluation metrics for the model:
    Accuracy, F1 Score, R Squre, RMSe
    """
    acc = 0
    f1score = 0
    rmse = 0
    r_sqaure = 0
    
    if len(y_pred) != len(y_actual):
        print('predicted and actual length not equal')
        
    else:
        len_y = len(y_pred)
        
        y_pred_bool =  y_pred >= 0
        y_actual_bool =  y_actual >= 0
        f1score = f1_score(y_actual_bool, y_pred_bool, average='binary')
        acc = accuracy_score(y_actual_bool, y_pred_bool)
        
        rmse = np.sqrt(mse(y_actual, y_pred))
        r_sqaure = r2_score(y_actual, y_pred)
        
    df_evaluation = pd.DataFrame({'Accuracy': pd.Series(acc),
                                 'F1 Score': pd.Series(f1score),
                                 'R Square': pd.Series(r_sqaure),
                                 'RMSE': pd.Series(rmse)})
    return(df_evaluation)

In [101]:
model_evaluation(pred, np.array(y_test))

,Accuracy,F1 Score,R Square,RMSE
0,0.631199,0.718125,0.1166,12.341792
